### Projeto Final de Spark
#### Campanha Nacional de Vacinação contra Covid-19

Autor: Arthur Freitas Gomes

Turma: Semantix Academy - Big Data Engineer 04-21

Professor: Rodrigo Rebouças

#### Enviar os dados para o hdfs

No terminal:

agomes:~/spark/input$ sudo mkdir desafio

agomes:~/spark/input/desafio$ sudo curl -O https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

agome:~/spark/input/desafio$ sudo unrar x 04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

agomes:~/spark/input/desafio$ docker exec -it namenode bash

    root@namenode:/# hdfs dfs -mkdir -p /user/arthur/desafio
    root@namenode:/# hdfs dfs -put /input/desafio/* /user/arthur/desafio
    root@namenode:/# hdfs dfs -ls /user/arthur/desafio

In [1]:
!hdfs dfs -ls /user/arthur/desafio

Found 5 items
-rw-r--r--   3 root supergroup   12445782 2021-07-08 09:17 /user/arthur/desafio/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar
-rw-r--r--   3 root supergroup   62492959 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup   76520681 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   3 root supergroup   91120916 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup    3046774 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv
